# 0.   Packages and Functions

In [1]:
!pip install pandas_datareader

In [2]:

#general
import pandas as pd
import numpy as np

#for dates and times
import random
import time
import datetime
from pandas.tseries.holiday import USFederalHolidayCalendar

#prices
import pandas_datareader as web


In [3]:


def str_time_prop(start, end, format, prop):
    """Get a time at a proportion of a range of two formatted times.

    start and end should be strings specifying times formated in the
    given format (strftime-style), giving an interval [start, end].
    prop specifies how a proportion of the interval to be taken after
    start.  The returned time will be in the specified format.
    """

    stime = time.mktime(time.strptime(start, format))
    etime = time.mktime(time.strptime(end, format))

    ptime = stime + prop * (etime - stime)

    return time.strftime(format, time.localtime(ptime))


def random_date(start, end, prop):
    return str_time_prop(start, end, '%Y-%m-%d', prop)

In [4]:
def weekend2weekday (transactions, col_date, col_day):


  dates2_BUY=list()
  days2_BUY=list()

  for index,row in transactions.iterrows():
    buy_date = datetime.datetime.strptime(row[col_date], "%Y-%m-%d")
    if row[col_day]=='Sat':
      buy_date2 = buy_date - datetime.timedelta(1)
      dates2_BUY.append(buy_date2)

      buy_day2='Fri'
      days2_BUY.append(buy_day2)
    elif row[col_day]=='Sun':
      buy_date = datetime.datetime.strptime(row[col_day], "%Y-%m-%d")
      buy_date2 = buy_date + datetime.timedelta(1)
      dates2_BUY.append(buy_date2)

      buy_day2='Mon'
      days2_BUY.append(buy_day2)
    else:
      buy_date2=row[col_date]
      dates2_BUY.append(buy_date2)
      buy_day2=row[col_day]
      days2_BUY.append(buy_day2)

    transactions['date_BUY_fix']=dates2_BUY
    transactions['day_BUY_fix']=days2_BUY


  return transactions

# 1. Setting investment parameters

## Temporal horizons
 In number of days
 - 1 day to 2 years

In [5]:
horizons=[1,2,3,4,5,6,7,10,15,20,30,40,50,60,70,80,90,100,120,150,180,210,240,270,300,330,360,420,480,540,600,660,720]


## Amounts in dollars
 - 50 to 50,000 dollars

In [6]:
amounts=[50,100,200,300,400,500,1000,1500,2000,3000,4000,5000,10000,15000,20000,25000,50000]

# 2. Data extraction

## Companies' stock prices

In [7]:
#Selecting a list of companies we are interested in analyzing:
companies = ['AAPL',
            'AMZN',
            'MSFT',
            'GOOG',
            'FB',
            'VISA',
            'NVDA',
            'PG',
            'TSLA']

We are going to extract info from 10 years from the companies selected

In [8]:
#Collecting the data of each company in a list
l_companies_prices=list()
error_companies=list()

for company in companies:
    try:
        prices= web.DataReader(company, data_source='yahoo', start='2010-10-10', end='2020-10-10')
        l_companies_prices.append(prices)
        print(company)
    
    except :
        print('There were  errors when extracting data of  ', company)
        #elimino de la lista esa empresa que no se consiguieron datos.
        error_companies.append(company)

for c in error_companies:
  companies.remove(c)
    

AAPL
AMZN
MSFT
GOOG
FB
There were  errors when extracting data of   VISA
NVDA
PG
TSLA


In [9]:
companies

['AAPL', 'AMZN', 'MSFT', 'GOOG', 'FB', 'NVDA', 'PG', 'TSLA']

In [10]:
len(l_companies_prices)

8

In [11]:
len(companies)

8

In [12]:
print(companies[0])
l_companies_prices[0].columns

AAPL


Index(['High', 'Low', 'Open', 'Close', 'Volume', 'Adj Close'], dtype='object')

In [13]:
company_pos=companies.index("AAPL")
l_companies_prices[company_pos].loc['2010-10-11','Close']

10.54857063293457

In [14]:
l_companies_prices[7].to_csv('PG_prices.csv')

In [15]:
l_companies_prices[7]

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2010-10-11,4.140000,4.014000,4.088000,4.048000,856000.0,4.048000
2010-10-12,4.056000,4.006000,4.040000,4.048000,1220000.0,4.048000
2010-10-13,4.170000,4.072000,4.128000,4.108000,1591000.0,4.108000
2010-10-14,4.206000,4.080000,4.200000,4.150000,1474000.0,4.150000
2010-10-15,4.180000,4.050000,4.178000,4.108000,1423500.0,4.108000
...,...,...,...,...,...,...
2020-10-05,433.640015,419.329987,423.350006,425.679993,44722800.0,425.679993
2020-10-06,428.779999,406.049988,423.790009,413.980011,49146300.0,413.980011
2020-10-07,429.899994,413.850006,419.869995,425.299988,43127700.0,425.299988


VOLATILITY

In [16]:
l_volatilities = []
for company in companies:
    company_pos=companies.index(company)
    df_company_prices = pd.DataFrame(l_companies_prices[company_pos])
    log_ret = np.log(df_company_prices['Close'] / df_company_prices['Close'].shift(1))
    volatility = log_ret.rolling(window=252).std() * np.sqrt(252)
    df_company_prices.insert(2, "Volatility", volatility, True)
    l_volatilities.append(df_company_prices)




In [17]:
print(l_volatilities)

[                  High         Low  Volatility        Open       Close  \
Date                                                                     
2010-10-11   10.615714   10.521429         NaN   10.526429   10.548571   
2010-10-12   10.696429   10.446071         NaN   10.550357   10.662143   
2010-10-13   10.784286   10.707143         NaN   10.721429   10.719286   
2010-10-14   10.802500   10.728571         NaN   10.774643   10.796786   
2010-10-15   11.250000   10.889643         NaN   10.980000   11.240714   
...                ...         ...         ...         ...         ...   
2020-10-05  116.650002  113.550003    0.444997  113.910004  116.500000   
2020-10-06  116.120003  112.250000    0.446135  115.699997  113.160004   
2020-10-07  115.550003  114.129997    0.446117  114.620003  115.080002   
2020-10-08  116.400002  114.589996    0.446045  116.250000  114.970001   
2020-10-09  117.000000  114.919998    0.446154  115.279999  116.970001   

                 Volume   Adj Close 

# 3. Random transactions

In [18]:
transactions=pd.DataFrame()
#we do 40,000 initial transactions

#id
transactions['id']=range(1,40001)

## Companies

In [19]:
#random company
transactions["company"]=np.random.choice(companies, size=len(transactions))

## Horizons

In [20]:
#random horizon
transactions["horizon"] = np.random.choice(horizons, size=len(transactions))


## Investment Amounts

In [21]:
#random investment amount
transactions["amount"] = np.random.choice(amounts, size=len(transactions))


## Dates - BUY

In [22]:
#random buy date
start='2013-10-10'
end='2018-10-10'

dates_buy=list()

for i in range(0, transactions.shape[0]):
    r_date=random_date(start,end,random.random())
    dates_buy.append(r_date)


print(len(dates_buy))
transactions['date_BUY']=dates_buy

40000


### Fixing dates (BUY)

In [23]:
day_names=list()

for d in dates_buy:

  dt = datetime.datetime.strptime(d, '%Y-%m-%d')
  weekno=dt.weekday()

  if weekno == 0:
    dia='Mon'
    day_names.append(dia)
  elif weekno==1:
    dia='Tue'
    day_names.append(dia)
  elif weekno==2:
    dia='Wed'
    day_names.append(dia)
  elif weekno==3:
    dia='Thu'
    day_names.append(dia)
  elif weekno==4:
    dia='Fri'
    day_names.append(dia)
  elif weekno==5:
    dia='Sat'
    day_names.append(dia)
  elif weekno==6:
    dia='Sun'
    day_names.append(dia)

transactions['day_BUY']=day_names


In [24]:
dates2_BUY=list()
days2_BUY=list()

#if date is saturday, we are going to push it to friday
#if date is sunday, we are going to push it to monday

for index,row in transactions.iterrows():
  buy_date = datetime.datetime.strptime(row['date_BUY'], "%Y-%m-%d")
  if row['day_BUY']=='Sat':
    buy_date2 = buy_date - datetime.timedelta(1)
    dates2_BUY.append(buy_date2)

    buy_day2='Fri'
    days2_BUY.append(buy_day2)
  elif row['day_BUY']=='Sun':
    buy_date = datetime.datetime.strptime(row['date_BUY'], "%Y-%m-%d")
    buy_date2 = buy_date + datetime.timedelta(1)
    dates2_BUY.append(buy_date2)

    buy_day2='Mon'
    days2_BUY.append(buy_day2)
  else:
    buy_date2=buy_date
    dates2_BUY.append(buy_date2)
    buy_day2=row['day_BUY']
    days2_BUY.append(buy_day2)


transactions['date_BUY_fix']=dates2_BUY
transactions['day_BUY_fix']=days2_BUY

# transactions=weekend2weekday(transactions,'date_BUY','day_BUY' )

In [25]:
#checking for holidays (NYSE is closed)

cal = USFederalHolidayCalendar()
holidays = cal.holidays(start='2010-10-10', end='2020-10-10').to_pydatetime()
# if datetime.datetime(2014,01,01) in holidays:
#     print True 

l_holidays=list()
for d in dates2_BUY:
  if d in holidays:
    holiday='Holiday'
    l_holidays.append(holiday)

  else:
    holiday=''
    l_holidays.append(holiday)

transactions['holiday_BUY']=l_holidays

In [26]:
transactions_h=transactions[transactions.holiday_BUY=='Holiday']

transactions=transactions[transactions.holiday_BUY!='Holiday']

## Prices - BUY

In [27]:
#price when bought

l_buy_prices=list()
l_volatility=list()
for index,row in transactions.iterrows():
  comp=row['company']
  buy_date=row['date_BUY_fix']
  company_pos=companies.index(comp)
  try:
    price=l_companies_prices[company_pos].loc[buy_date,'Adj Close']
    volatility= l_volatilities[company_pos].loc[buy_date, 'Volatility']
  except :
    price=0
    

  l_buy_prices.append(price)

transactions['price_BUY']=l_buy_prices


VOLATILITY BUY


In [28]:
l_volatility_buy=list()
for index,row in transactions.iterrows():
  comp=row['company']
  buy_date=row['date_BUY_fix']
  company_pos=companies.index(comp)
  try:
    volatility= l_volatilities[company_pos].loc[buy_date, 'Volatility']
  except :
    volatility=0
  l_volatility_buy.append(volatility)
print(len(l_volatility_buy))


transactions['Volatility_Buy'] = l_volatility_buy

38017


## Dates - SELL

In [29]:
#random sell date
dates_SELL=list()

for index,row in transactions.iterrows():

  buy_date = row['date_BUY_fix']
  sell_date = buy_date + datetime.timedelta(row['horizon'])
  
  dates_SELL.append(sell_date)

transactions['date_SELL']=dates_SELL

### Fixing dates (SELL)

In [30]:
day_names=list()



for d in dates_SELL:

  # dt = datetime.datetime.strptime(d, '%Y-%m-%d')
  weekno=d.weekday()

  if weekno == 0:
    dia='Mon'
    day_names.append(dia)
  elif weekno==1:
    dia='Tue'
    day_names.append(dia)
  elif weekno==2:
    dia='Wed'
    day_names.append(dia)
  elif weekno==3:
    dia='Thu'
    day_names.append(dia)
  elif weekno==4:
    dia='Fri'
    day_names.append(dia)
  elif weekno==5:
    dia='Sat'
    day_names.append(dia)
  elif weekno==6:
    dia='Sun'
    day_names.append(dia)

transactions['day_SELL']=day_names


In [31]:
dates2_SELL=list()
days2_SELL=list()

#if date is saturday, we are going to push it to friday
#if date is sunday, we are going to push it to monday

for index,row in transactions.iterrows():
  SELL_date = row['date_SELL']
  if row['day_SELL']=='Sat':
    SELL_date2 = SELL_date - datetime.timedelta(1)
    dates2_SELL.append(SELL_date2)

    SELL_day2='Fri'
    days2_SELL.append(SELL_day2)
  elif row['day_SELL']=='Sun':
    SELL_date = row['date_SELL']
    SELL_date2 = SELL_date + datetime.timedelta(1)
    dates2_SELL.append(SELL_date2)

    SELL_day2='Mon'
    days2_SELL.append(SELL_day2)
  else:
    SELL_date2=SELL_date
    dates2_SELL.append(SELL_date2)
    SELL_day2=row['day_SELL']
    days2_SELL.append(SELL_day2)


transactions['date_SELL_fix']=dates2_SELL
transactions['day_SELL_fix']=days2_SELL

# transactions=weekend2weekday(transactions,'date_BUY','day_BUY' )

In [32]:
transactions

,id,company,horizon,amount,date_BUY,day_BUY,date_BUY_fix,day_BUY_fix,holiday_BUY,price_BUY,Volatility_Buy,date_SELL,day_SELL,date_SELL_fix,day_SELL_fix
0,1,PG,70,500,2016-07-05,Tue,2016-07-05,Tue,,74.794472,0.166181,2016-09-13,Tue,2016-09-13,Tue
1,2,TSLA,4,300,2014-02-23,Sun,2014-02-24,Mon,,43.529999,0.684741,2014-02-28,Fri,2014-02-28,Fri
2,3,NVDA,70,5000,2015-05-15,Fri,2015-05-15,Fri,,20.579834,0.273231,2015-07-24,Fri,2015-07-24,Fri
4,5,AMZN,3,1500,2015-08-15,Sat,2015-08-14,Fri,,531.520020,0.308135,2015-08-17,Mon,2015-08-17,Mon
5,6,FB,7,20000,2017-03-07,Tue,2017-03-07,Tue,,137.300003,0.196505,2017-03-14,Tue,2017-03-14,Tue
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,39996,PG,240,4000,2015-01-29,Thu,2015-01-29,Thu,,71.914757,0.122401,2015-09-26,Sat,2015-09-25,Fri
39996,39997,PG,6,1000,2014-07-19,Sat,2014-07-18,Fri,,66.628288,0.122396,2014-07-24,Thu,2014-07-24,Thu
39997,39998,NVDA,300,300,2017-04-29,Sat,2017-04-28,Fri,,103.163094,0.447867,2018-02-22,Thu,2018-02-22,Thu
39998,39999,PG,5,200,2015-02-19,Thu,2015-02-19,Thu,,71.528603,0.120155,2015-02-24,Tue,2015-02-24,Tue


In [33]:
#checking for holidays (NYSE is closed)
cal = USFederalHolidayCalendar()
holidays = cal.holidays(start='2010-10-10', end='2020-10-10').to_pydatetime()
# if datetime.datetime(2014,01,01) in holidays:
#     print True 

l_holidays=list()
for d in dates2_SELL:
  if d in holidays:
    holiday='Holiday'
    l_holidays.append(holiday)

  else:
    holiday=''
    l_holidays.append(holiday)

transactions['holiday_SELL']=l_holidays

## Prices - SELL

In [34]:
#price when sold

l_sell_prices=list()
for index,row in transactions.iterrows():
  comp=row['company']
  sell_date=row['date_SELL_fix']
  company_pos=companies.index(comp)
  try:
    price=l_companies_prices[company_pos].loc[sell_date,'Adj Close']
  except :
    price=0

  l_sell_prices.append(price)

transactions['price_SELL']=l_sell_prices


VOLATILITY SELL

In [35]:
l_volatility_sell=list()
for index,row in transactions.iterrows():
  comp=row['company']
  sell_date=row['date_SELL_fix']
  company_pos=companies.index(comp)
  try:
    volatility=l_volatilities[company_pos].loc[sell_date,'Volatility']
  except :
    volatility=0

  l_volatility_sell.append(volatility)

transactions['Volatility_sell']=l_volatility_sell

In [36]:
transactions_h=transactions[transactions.holiday_SELL=='Holiday']

transactions=transactions[transactions.holiday_SELL!='Holiday']

In [37]:
transactions

,id,company,horizon,amount,date_BUY,day_BUY,date_BUY_fix,day_BUY_fix,holiday_BUY,price_BUY,Volatility_Buy,date_SELL,day_SELL,date_SELL_fix,day_SELL_fix,holiday_SELL,price_SELL,Volatility_sell
0,1,PG,70,500,2016-07-05,Tue,2016-07-05,Tue,,74.794472,0.166181,2016-09-13,Tue,2016-09-13,Tue,,76.801003,0.144486
1,2,TSLA,4,300,2014-02-23,Sun,2014-02-24,Mon,,43.529999,0.684741,2014-02-28,Fri,2014-02-28,Fri,,48.962002,0.694409
2,3,NVDA,70,5000,2015-05-15,Fri,2015-05-15,Fri,,20.579834,0.273231,2015-07-24,Fri,2015-07-24,Fri,,18.850248,0.280573
4,5,AMZN,3,1500,2015-08-15,Sat,2015-08-14,Fri,,531.520020,0.308135,2015-08-17,Mon,2015-08-17,Mon,,535.219971,0.308176
5,6,FB,7,20000,2017-03-07,Tue,2017-03-07,Tue,,137.300003,0.196505,2017-03-14,Tue,2017-03-14,Tue,,139.320007,0.195243
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,39996,PG,240,4000,2015-01-29,Thu,2015-01-29,Thu,,71.914757,0.122401,2015-09-26,Sat,2015-09-25,Fri,,61.996246,0.155721
39996,39997,PG,6,1000,2014-07-19,Sat,2014-07-18,Fri,,66.628288,0.122396,2014-07-24,Thu,2014-07-24,Thu,,66.388382,0.121443
39997,39998,NVDA,300,300,2017-04-29,Sat,2017-04-28,Fri,,103.163094,0.447867,2018-02-22,Thu,2018-02-22,Thu,,240.289780,0.417705
39998,39999,PG,5,200,2015-02-19,Thu,2015-02-19,Thu,,71.528603,0.120155,2015-02-24,Tue,2015-02-24,Tue,,71.772064,0.120321


In [38]:
transactions_ready=transactions[['company','horizon','amount','date_BUY_fix','date_SELL_fix','price_BUY','price_SELL']]

In [39]:
transactions_ready

,company,horizon,amount,date_BUY_fix,date_SELL_fix,price_BUY,price_SELL
0,PG,70,500,2016-07-05,2016-09-13,74.794472,76.801003
1,TSLA,4,300,2014-02-24,2014-02-28,43.529999,48.962002
2,NVDA,70,5000,2015-05-15,2015-07-24,20.579834,18.850248
4,AMZN,3,1500,2015-08-14,2015-08-17,531.520020,535.219971
5,FB,7,20000,2017-03-07,2017-03-14,137.300003,139.320007
...,...,...,...,...,...,...,...
39995,PG,240,4000,2015-01-29,2015-09-25,71.914757,61.996246
39996,PG,6,1000,2014-07-18,2014-07-24,66.628288,66.388382
39997,NVDA,300,300,2017-04-28,2018-02-22,103.163094,240.289780
39998,PG,5,200,2015-02-19,2015-02-24,71.528603,71.772064


## Filtering transactions with error (Price = 0)

In [40]:
price_error=transactions_ready[(transactions_ready.price_BUY==0) | (transactions_ready.price_SELL==0)]
print('with errors: ', price_error.shape)
print(transactions_ready.shape)

transactions_ready=pd.concat([transactions_ready, price_error, price_error]).drop_duplicates(keep=False)
print(transactions_ready.shape)

with errors:  (439, 7)
(36180, 7)
(35481, 7)


# Real Profit/Loss (Rentabilidad Real)

In [41]:
transactions_ready['nominal_return']=(transactions_ready['price_SELL']-transactions_ready['price_BUY'])/transactions_ready['price_BUY']

In [42]:
transactions_ready

,company,horizon,amount,date_BUY_fix,date_SELL_fix,price_BUY,price_SELL,nominal_return
0,PG,70,500,2016-07-05,2016-09-13,74.794472,76.801003,0.026827
1,TSLA,4,300,2014-02-24,2014-02-28,43.529999,48.962002,0.124788
2,NVDA,70,5000,2015-05-15,2015-07-24,20.579834,18.850248,-0.084043
4,AMZN,3,1500,2015-08-14,2015-08-17,531.520020,535.219971,0.006961
5,FB,7,20000,2017-03-07,2017-03-14,137.300003,139.320007,0.014712
...,...,...,...,...,...,...,...,...
39995,PG,240,4000,2015-01-29,2015-09-25,71.914757,61.996246,-0.137920
39996,PG,6,1000,2014-07-18,2014-07-24,66.628288,66.388382,-0.003601
39997,NVDA,300,300,2017-04-28,2018-02-22,103.163094,240.289780,1.329222
39998,PG,5,200,2015-02-19,2015-02-24,71.528603,71.772064,0.003404


In [43]:
transactions_ready['investment']=''
investments=list()
for index, row in transactions_ready.iterrows():
  if row['nominal_return']>0:
    inv='GOOD'
    investments.append(inv)
  elif row['nominal_return']<0:
    inv='BAD'
    investments.append(inv)
  else:
    inv='0'
    investments.append(inv)

transactions_ready['investment']=investments

In [44]:
transactions_ready.shape

(35481, 9)

In [45]:
transactions_ready

,company,horizon,amount,date_BUY_fix,date_SELL_fix,price_BUY,price_SELL,nominal_return,investment
0,PG,70,500,2016-07-05,2016-09-13,74.794472,76.801003,0.026827,GOOD
1,TSLA,4,300,2014-02-24,2014-02-28,43.529999,48.962002,0.124788,GOOD
2,NVDA,70,5000,2015-05-15,2015-07-24,20.579834,18.850248,-0.084043,BAD
4,AMZN,3,1500,2015-08-14,2015-08-17,531.520020,535.219971,0.006961,GOOD
5,FB,7,20000,2017-03-07,2017-03-14,137.300003,139.320007,0.014712,GOOD
...,...,...,...,...,...,...,...,...,...
39995,PG,240,4000,2015-01-29,2015-09-25,71.914757,61.996246,-0.137920,BAD
39996,PG,6,1000,2014-07-18,2014-07-24,66.628288,66.388382,-0.003601,BAD
39997,NVDA,300,300,2017-04-28,2018-02-22,103.163094,240.289780,1.329222,GOOD
39998,PG,5,200,2015-02-19,2015-02-24,71.528603,71.772064,0.003404,GOOD


In [46]:
transactions_ready.investment.value_counts()

GOOD    24662
BAD     10478
0         341
Name: investment, dtype: int64

## Variables calculation

### Expected return

In [47]:
l_returns=list()
l_stdDev=list()

for company in companies:
  company_pos=companies.index(company)
  prices_r=l_companies_prices[company_pos]['Adj Close']

  # Calculting log returns and std deviation
  percent_change = prices_r.pct_change()
  returns = np.log(1+percent_change)

  meanReturn_daily = returns.mean()
  stdDev_daily = returns.std()    

  meanReturn_year= meanReturn_daily * 365
  stdDev_year= stdDev_daily *stdDev_daily* 365

  l_returns.append(meanReturn_year)
  l_stdDev.append(stdDev_year)
  


In [48]:
companies_returns=list(zip(companies,l_returns))
companies_stddev=list(zip(companies,l_stdDev))

In [49]:
companies_returns

[('AAPL', 0.37006814429896073),
 ('AMZN', 0.4447562102652766),
 ('MSFT', 0.34874318892447237),
 ('GOOG', 0.2509879900028765),
 ('FB', 0.33424316922218367),
 ('NVDA', 0.582100040715823),
 ('PG', 0.16589283819663966),
 ('TSLA', 0.6779141428282532)]

In [50]:
#prueba de modificacion desde jupyter notebook.

Expected return 1
